In [ ]:
import json
import redis
import os
from dotenv import load_dotenv
load_dotenv()
from redisvl.index import SearchIndex

In [ ]:
r = redis.Redis(host=os.environ.get('host'), port=os.environ.get('port'), decode_responses=True)
p = r.pipeline()
if r.ping() : print('Connected to Redis')
r.flushdb()

In [ ]:
schema = {
    "index": {
        "name": "idx:users",
        "prefix": "user",
        "storage_type": "json"
    },
    "fields": [
        {"name": "userid", "type": "numeric"},
        {"name": "username", "type": "tag"},
        {"name": "categories_purchased", "type": "tag", "attrs" : {"seperator" : ","}}]
}

index = SearchIndex.from_dict(schema)
index.set_client(r)
index.create(overwrite=True)

In [ ]:
with open('users.json', "r") as file:
    data = json.load(file)
for user in data:
    user['categories_purchased'] = ",".join(user['categories_purchased'])
keys  = index.load(data)

In [ ]:
# FT.TAGVALS 'idx:users' categories_purchased

idx_products = r.ft("idx:users")
categories = idx_products.tagvals('categories_purchased')
print(categories)

In [ ]:
# Target all users who have purchased electronics & add them to a cuckoo filter
r.cf().reserve(key='target_users', capacity=1000000)

idx_products = r.ft("idx:users")
results = idx_products.search(query="@categories_purchased:{electronics}")
# print(results)

# Iterate through the results
for doc in results.docs:
    # print(f"Document ID: {doc.id}")
    for field, value in doc.__dict__.items():
        if field.startswith('json'):  # Exclude internal attributes
            value = json.loads(value)
            print(value['userid'])
            r.cf().add(key='target_users',item=value['userid'])

In [ ]:
# A user logs in

incoming_user_id = 50

result = r.cf().exists(key="target_users", item=incoming_user_id)

if result:
    print("The user may be targetted, show the ad")
    print(f"Assume, the incoming user {incoming_user_id} signs up, then delete the user from cuckoo filter")
    r.cf().delete(key='target_users', item=incoming_user_id)
else:
    print(f"The user {incoming_user_id} is not targetted or has already signed up for the campaign")

In [ ]:
result = r.cf().info(key='target_users')

print(result['insertedNum'])